In [ ]:
!pip install pyradiomics
import yaml
from radiomics import featureextractor
import pandas as pd
import os

In [ ]:

# Gốc dataset
base_dir = "/kaggle/input/your_dataset"

# Cấu trúc folder
splits = {
    "train": {"images": "train_images", "masks": "train_masks"},
    "test": {"images": "test_images", "masks": "test_masks"},
    "valid": {"images": "valid_images", "masks": "valid_masks"}
}

rows = []

for split, dirs in splits.items():
    img_dir = os.path.join(base_dir, dirs["images"])
    mask_dir = os.path.join(base_dir, dirs["masks"])

    # duyệt qua toàn bộ file trong folder ảnh
    for f in os.listdir(img_dir):
        if f.endswith(".nii.gz") or f.endswith(".nii"):  # nhận cả .nii và .nii.gz
            patient_id = os.path.splitext(os.path.splitext(f)[0])[0]
            img_path = os.path.join(img_dir, f)
            mask_path = os.path.join(mask_dir, f)

            if os.path.exists(mask_path):
                rows.append({
                    "PatientID": patient_id,
                    "Image": img_path,
                    "Mask": mask_path,
                    "Split": split
                })
            else:
                print(f"⚠️ Không tìm thấy mask cho {f} ({split})")

# Xuất CSV
df = pd.DataFrame(rows)
df.to_csv("dataset_1.csv", index=False)

print(f"✅ Tạo file dataset.csv với {len(df)} dòng")


In [ ]:

# Gốc dataset
base_dir = "/kaggle/input/your_dataset"

# Cấu trúc folder
splits = {
    "train": {"images": "train_images", "masks": "train_masks"},
    "test": {"images": "test_images", "masks": "test_masks"},
    "valid": {"images": "valid_images", "masks": "valid_masks"}
}

rows = []

for split, dirs in splits.items():
    img_dir = os.path.join(base_dir, dirs["images"])
    mask_dir = os.path.join(base_dir, dirs["masks"])

    # duyệt qua toàn bộ file trong folder ảnh
    for f in os.listdir(img_dir):
        if f.endswith(".nii.gz") or f.endswith(".nii"):  # nhận cả .nii và .nii.gz
            patient_id = os.path.splitext(os.path.splitext(f)[0])[0]
            img_path = os.path.join(img_dir, f)
            mask_path = os.path.join(mask_dir, f)

            if os.path.exists(mask_path):
                rows.append({
                    "PatientID": patient_id,
                    "Image": img_path,
                    "Mask": mask_path,
                    "Split": split
                })
            else:
                print(f"⚠️ Không tìm thấy mask cho {f} ({split})")

# Xuất CSV
df = pd.DataFrame(rows)
df.to_csv("dataset_2.csv", index=False)

print(f"✅ Tạo file dataset.csv với {len(df)} dòng")


In [ ]:
config_dict = {
    "imageType": {
        "Original": {},
        "Wavelet": {},
        "LoG": {"sigma": [1.0, 2.0, 3.0]}  # Lọc Gaussian Laplace
    },
    "featureClass": {
        "shape": None,
        "firstorder": None,
        "glcm": None,
        "glrlm": None,
        "glszm": None,
        "gldm": None,
        "ngtdm": None
    },
    "setting": {
        "resampledPixelSpacing": [1, 1, 1],  # Chuẩn hóa voxel spacing
        "interpolator": "sitkBSpline",
        "binWidth": 25,         # Quantization bin width
        "padDistance": 5,
        "correctMask": True
    },
}
# Lưu config
with open("config_mri.yaml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)
extractor = featureextractor.RadiomicsFeatureExtractor("config_mri.yaml")
df = pd.read_csv("dataset_1.csv")
all_features = []
for idx, row in df.iterrows():
    try:
        features = extractor.execute(row["Image"], row["Mask"])
        # Giữ lại các feature bắt đầu bằng "original"
        feature_dict = {k: v for k, v in features.items() if k.startswith("original")}
        # Thêm thông tin ID, Split
        feature_dict["PatientID"] = row["PatientID"]
        feature_dict["Split"] = row["Split"]
        all_features.append(feature_dict)
        print(f"✅ Done {row['PatientID']} ({row['Split']})")
    except Exception as e:
        print(f"❌ Error {row['PatientID']}: {e}")
features_df = pd.DataFrame(all_features)
features_df.to_csv("features_mri_1.csv", index=False)
print("🎉 Features saved to features_mri_1.csv")

In [ ]:
config_dict = {
    "imageType": {
        "Original": {},
        "Wavelet": {},
        "LoG": {"sigma": [1.0, 2.0, 3.0]}  # Lọc Gaussian Laplace
    },
    "featureClass": {
        "shape": None,
        "firstorder": None,
        "glcm": None,
        "glrlm": None,
        "glszm": None,
        "gldm": None,
        "ngtdm": None
    },
    "setting": {
        "resampledPixelSpacing": [1, 1, 1],  # Chuẩn hóa voxel spacing
        "interpolator": "sitkBSpline",
        "binWidth": 25,         # Quantization bin width
        "padDistance": 5,
        "correctMask": True
    },
}
# Lưu config
with open("config_mri.yaml", "w") as f:
    yaml.dump(config_dict, f, default_flow_style=False)
extractor = featureextractor.RadiomicsFeatureExtractor("config_mri.yaml")
df = pd.read_csv("dataset_2.csv")
all_features = []
for idx, row in df.iterrows():
    try:
        features = extractor.execute(row["Image"], row["Mask"])
        # Giữ lại các feature bắt đầu bằng "original"
        feature_dict = {k: v for k, v in features.items() if k.startswith("original")}
        # Thêm thông tin ID, Split
        feature_dict["PatientID"] = row["PatientID"]
        feature_dict["Split"] = row["Split"]
        all_features.append(feature_dict)
        print(f"✅ Done {row['PatientID']} ({row['Split']})")
    except Exception as e:
        print(f"❌ Error {row['PatientID']}: {e}")
features_df = pd.DataFrame(all_features)
features_df.to_csv("features_mri_2.csv", index=False)
print("🎉 Features saved to features_mri_2.csv")